In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.common.exceptions import WebDriverException, TimeoutException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from selenium.common.exceptions import StaleElementReferenceException
import pandas as pd, numpy as np
from pandas.io.formats.excel import ExcelFormatter
import os
import nbimporter
from datetime import datetime
from main_collection import raw_cleaner
from main_collection import idx_find
from main_collection import find_date_format
from main_collection import find_lig
from main_collection import fn_acc_record
from main_collection import find_IYScore
import warnings
import re
warnings.filterwarnings('ignore')
from openpyxl import load_workbook

class BadGatewayException(Exception):
    pass  

In [2]:
from datetime import datetime, timedelta

def get_date_range():
    dates=[]
    
    start_date = input('Entry START DATE - Format:  dd-mm-yyyy')

    # Kullanıcıdan başlangıç tarihini iste
    start_year = int(start_date.split('-')[2])
    start_month = int(start_date.split('-')[1])
    start_day = int(start_date.split('-')[0])

    # Kullanıcıdan bitiş tarihini iste
    end_date = input('Entry END DATE - Format:  dd-mm-yyyy')
    
    end_year = int(end_date.split('-')[2])
    end_month = int(end_date.split('-')[1])
    end_day = int(end_date.split('-')[0])

    # Başlangıç ve bitiş tarihlerini kullanarak işlemleri gerçekleştir
    start_date = datetime(start_year, start_month, start_day)

    end_date = datetime(end_year, end_month, end_day)

    # Tarih aralığını hesapla
    date_range = end_date - start_date

    # Günleri liste olarak tut
    date_list = [start_date + timedelta(days=x) for x in range(date_range.days + 1)]

    # Liste üzerindeki her bir tarihi isteğe bağlı olarak yazdır
    for target_date in date_list:
        dates.append(target_date.strftime('%d-%m-%Y'))
    print(dates)
        
    return dates

In [3]:
def accept_cookies(driver):
    wait = WebDriverWait(driver, 30)
    cookies = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button#didomi-notice-agree-button')))
    cookies.click()

In [4]:
def get_current_year_month(driver):
    wait = WebDriverWait(driver, 30)
    current_month = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "body > div.page-container.page-container--legacy-link-banner-visible > div > main > div.page-livescore-index > div.page-livescore-index__container > div.widget-stencilbar-livescore > div.widget-stencilbar-livescore__header > div.widget-dateslider > div.widget-dateslider__datepicker > div > div.widget-datepicker__header > div.widget-datepicker__selector.widget-datepicker__selector--month > div.widget-datepicker__value")))
    month_str = current_month.text.strip()[:3]
    month_mapping = {'OCA': 1, 'ŞUB': 2, 'MAR': 3, 'NIS': 4, 'MAY': 5, 'HAZ': 6, 'TEM': 7, 'AĞU': 8, 'EYL': 9, 'EKI': 10, 'KAS': 11, 'ARA': 12}
    current_month_numeric = month_mapping.get(month_str)

    current_year_element = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'body > div.page-container.page-container--legacy-link-banner-visible > div > main > div.page-livescore-index > div.page-livescore-index__container > div.widget-stencilbar-livescore > div.widget-stencilbar-livescore__header > div.widget-dateslider > div.widget-dateslider__datepicker > div > div.widget-datepicker__header > div.widget-datepicker__selector.widget-datepicker__selector--year > div.widget-datepicker__value')))
    current_year = int(current_year_element.text.strip())

    return current_year, current_month_numeric

In [5]:
def set_target_date(driver, target_year, target_month, target_day):
    wait = WebDriverWait(driver, 30)
    dateIcon = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.widget-dateslider__datepicker-toggle')))
    

    # Click on the element
    dateIcon.click() 
    
    previous_year_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "body > div.page-container.page-container--legacy-link-banner-visible > div > main > div.page-livescore-index > div.page-livescore-index__container > div.widget-stencilbar-livescore > div.widget-stencilbar-livescore__header > div.widget-dateslider > div.widget-dateslider__datepicker.widget-dateslider__datepicker--open > div > div.widget-datepicker__header > div.widget-datepicker__selector.widget-datepicker__selector--year > div.widget-datepicker__nav.widget-datepicker__nav--previous")))
    previous_month_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "body > div.page-container.page-container--legacy-link-banner-visible > div > main > div.page-livescore-index > div.page-livescore-index__container > div.widget-stencilbar-livescore > div.widget-stencilbar-livescore__header > div.widget-dateslider > div.widget-dateslider__datepicker.widget-dateslider__datepicker--open > div > div.widget-datepicker__header > div.widget-datepicker__selector.widget-datepicker__selector--month > div.widget-datepicker__nav.widget-datepicker__nav--previous")))

    current_year, current_month = get_current_year_month(driver)

    while current_year > target_year or (current_year == target_year and current_month > target_month):
        previous_year_button.click()
        #time.sleep(1)
        current_year, current_month = get_current_year_month(driver)

    while current_month != target_month:
        previous_month_button.click()
        #time.sleep(1)
        current_year, current_month = get_current_year_month(driver)

    day_element = driver.find_element(By.XPATH, f"//td[contains(@class, 'widget-datepicker__calendar-body-cell') and text()='{target_day}']")
    day_element.click()
    time.sleep(7)

In [6]:
def process_links(driver, wait_time=15, max_iterations=2):
    count = 0
    iteration_count = 0
   
    
    while True:
        try:
            # Container elementini bul
            container_css = "div.match-row__match-content"
            all_links_elements = WebDriverWait(driver, wait_time).until(
                EC.visibility_of_all_elements_located((By.CSS_SELECTOR, container_css))
            )
            
            print(f'Total number of links: {len(all_links_elements)}')

            # Iterate through each element and click
            for i, element in enumerate(all_links_elements, start=1):
                result = []
                e = None

                try:
                    print(f"(try Worked... {str(i)})")
                    
                    # Precautionary for page loading
                    dynamic_xpath = f"(//div[@class='match-row__match-content'])[{i}]"
                    #elementx = WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, dynamic_xpath)))

                    max_retry_count = 5
                    retry_delay = 5  # saniye cinsinden

                    for _ in range(max_retry_count):
                        try:
                            element.click()
                            break  # If the click is successful, exit the loop
                        except (WebDriverException, TimeoutException, BadGatewayException) as e:
                            if "502 Bad Gateway" in str(e):
                                print("502 Bad Gateway error. Retrying...")
                                break
                            print(f"Error occurred: {e}")
                            time.sleep(retry_delay)

                    if "502 Bad Gateway" in str(e):
                        # Close the current tab and switch back to the original tab
                        driver.close()
                        driver.switch_to.window(driver.window_handles[0])
                        continue

                    # Switch to the newly opened tab (assuming it's the last tab)
                    driver.switch_to.window(driver.window_handles[-1])
                    print(f"(went new page... {str(i)})")

                    # Perform actions in the new tab
                    iddia_button = WebDriverWait(driver, wait_time).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, "[data-item-id='iddaa']"))
                    )

                    # get class attribute values...
                    tx = iddia_button.get_attribute('class')

                    # Check if the iddia_button is disabled
                    if "disabled" not in tx:
                        print('Button is not disabled. Proceeding...')
                        iddia_button.click()
                        time.sleep(1)

                        # Top scoreboard table element
                        mac_scorboard = WebDriverWait(driver, wait_time).until(
                            EC.element_to_be_clickable((By.CSS_SELECTOR, "div.p0c-soccer-match-details-header"))
                        )

                        # Main betting table element
                        mac_details_data = WebDriverWait(driver, wait_time).until(
                            EC.element_to_be_clickable((By.CSS_SELECTOR, ".widget-iddaa-markets__markets-list"))
                        )

                        # Scoreboard as a string
                        scoreboard_text = mac_scorboard.text
                        lines = scoreboard_text.strip().split('\n')
                        for line_sb in lines:
                            result += [line_sb]
                        print(f'-------------------\n{result}')

                        # Get match details as string text
                        mac_details_data_text = mac_details_data.text
                        detail_lines = mac_details_data_text.strip().split('\n')  # detail_lines is a list

                        for line_md in detail_lines:
                            result += [line_md]

                        crude_df = pd.DataFrame(result, columns=[0])
                        #crude_df.to_excel(f'../data/new_data_raw_{i}_.xlsx')
                        cleaned_df = raw_cleaner(list(crude_df[0]))
                        fn_acc_record('../data/df_all_cleaned_games.xlsx', cleaned_df, header=False, index=False)

                        # Close the current tab
                        driver.close()
                        print(f"(gone new page closed.. {str(i)})")
                        # Switch back to the original tab
                        driver.switch_to.window(driver.window_handles[0])

                        time.sleep(1)
                    else:
                        print('Button is disabled. Skipping...')
                        # Switch back to the original tab
                        driver.close()
                        driver.switch_to.window(driver.window_handles[0])

                except StaleElementReferenceException:
                    print('<><><><><><', count)
                    print("Stale Element Reference Exception. Retrying.")
                    continue
                    
                if  i==len(all_links_elements):
                    print('all match raw_links r collected')
                    return   
                    
            iteration_count += 1
            # Check if it's time to restart the loop
            if iteration_count % max_iterations == 0:
                print(f"Restarting the loop after {max_iterations} iterations.")
                break
                
            if  i==len(all_links_elements):
                print('all match raw_links r collected')
                raise StopIteration    
    
        except StaleElementReferenceException:
            print("Stale Element Reference Exception during initial find. Retrying after refreshing the page.")
            driver.refresh()
            continue
            
         

In [7]:
def main_function(driver):
    
   
    macKolikUrl = "https://www.mackolik.com/futbol/canli-sonuclar"
    driver.get(macKolikUrl)
    # date_list = ['10-10-2020', '11-10-2020', '12-10-2020', '13-10-2020', '14-10-2020', '15-10-2020', '16-10-2020', '17-10-2020']
    accept_cookies(driver)
    date_list = get_date_range()
    while True:
        

        while date_list:
            
            print('trueeeeeeeeeeeeeeeeeeeeeeee<<><><><><><><<><><><>><>')
            target_date = date_list.pop(0)  # Listenin başındaki tarihi al ve listeden çıkar
            print(target_date)
            print('tarih secildi*****************************************************')
            tarih_obj = datetime.strptime(target_date, '%d-%m-%Y')

            target_day = tarih_obj.day
            target_month = tarih_obj.month
            target_year = tarih_obj.year

            set_target_date(driver, target_year, target_month, target_day)

            #wait_for_page_loaded(driver)

            # 'process_links' 
            process_links(driver)

In [8]:
geckodriver_path = "../set/geckodriver"
# Firefox service nesnesini oluştur
service = FirefoxService(executable_path=geckodriver_path)
macKolikUrl = "https://www.mackolik.com/futbol/canli-sonuclar"
# Firefox tarayıcısını service ile başlat
driver = webdriver.Firefox(service=service)


main_function(driver)  #1-12-2023

Entry START DATE - Format:  dd-mm-yyyy 1-12-2023
Entry END DATE - Format:  dd-mm-yyyy 2-12-2023


['01-12-2023', '02-12-2023']
trueeeeeeeeeeeeeeeeeeeeeeee<<><><><><><><<><><><>><>
01-12-2023
tarih secildi*****************************************************
Total number of links: 244
(try Worked... 1)
(went new page... 1)
Button is not disabled. Proceeding...
-------------------
['Hatayspor', 'Antalyaspor', '3', 'MS', '3', '(İY 1 - 1)', '01-12-23 | Süper Lig | 14. Hafta | Stat: Mersin Stadyumu (3760)']
(gone new page closed.. 1)
(try Worked... 2)
(went new page... 2)
Button is not disabled. Proceeding...
-------------------
['Karagümrük', 'İstanbulspor', '3', 'MS', '0', '(İY 1 - 0)', '01-12-23 | Süper Lig | 14. Hafta | Stat: Atatürk Olimpiyat Stadı (310)']
(gone new page closed.. 2)
(try Worked... 3)
(went new page... 3)
Button is not disabled. Proceeding...
-------------------
['Adanaspor', 'Çorum FK', '0', 'MS', '3', '(İY 0 - 2)', '01-12-23 | TFF 1. Lig | 14. Hafta | Stat: Yeni Adana Stadyumu']
(gone new page closed.. 3)
(try Worked... 4)
(went new page... 4)
Button is not disabl

KeyboardInterrupt: 